In [2]:
import xgboost as xgb
import pandas as pd
import numpy as np
from sklearn.datasets import dump_svmlight_file
import ml_metrics

In [3]:
dfTrain = pd.read_csv('data/train.csv')
dfTest = pd.read_csv('data/test.csv')

In [4]:
#predictors = ['x', 'y', 'accuracy', 'time']
#X = dfTrain[predictors]
#y = dfTrain.place_id
#dump_svmlight_file(X, y,'data/train.libfm',zero_based=True,multilabel=False)
#len(dfTrain.place_id.unique())
print dfTrain.place_id.head(20) 
        

0     8523065625
1     1757726713
2     1137537235
3     6567393236
4     7440663949
5     6289802927
6     9931249544
7     5662813655
8     8471780938
9     1253803156
10    8684462954
11    2159916487
12    7652380351
13    8234363596
14    2272949794
15    4740742194
16    2123587484
17    8016758016
18    8936085695
19    2778700985
Name: place_id, dtype: int64


In [3]:
dictLabelIndex = {}
def genLabel(lstLabel):
    # The lalel need to be 0 to num_class - 1
    index = 0
    for label in lstLabel:
        if not dictLabelIndex.has_key(label):
            dictLabelIndex[label] = index
            index += 1
    lstIndex = []
    for label in lstLabel:
        index = dictLabelIndex[label]
        lstIndex.append(index)
    return lstIndex

def getLabel(lstLabel):
    lstIndex = []
    for label in lstLabel:
        if dictLabelIndex.has_key(label):
            index = dictLabelIndex[label]
        else:
            index = 0
        lstIndex.append(index)
    return lstIndex

In [ ]:
# label need to be 0 to num_class -1
data = np.loadtxt('data/train.csv.100000', delimiter=',',skiprows = 1, 
                  converters={})

sz = data.shape

print len(set(data[:, 5]))

genLabel(data[:, 5])
#print 'dictLabelIndex', dictLabelIndex

train = data[:int(len(data) * 0.7), :]
test = data[int(len(data) * 0.7):, :]

train_X = train[:,1:5]
train_Y = np.array(getLabel(train[:, 5]))
print 'train_Y', train_Y
print 'train_X', train_X
#print train_Y

test_X = test[:,1:5]
test_Y = np.array(getLabel(test[:, 5]))

xg_train = xgb.DMatrix( train_X, label = train_Y)
xg_test = xgb.DMatrix(test_X, label = test_Y)
num_class = len(set(data[:, 5]))
print 'num_class', num_class
# setup parameters for xgboost
param = {}
# use softmax multi-class classification
param['objective'] = 'multi:softmax'
# scale weight of positive examples
param['eta'] = 0.1
param['max_depth'] = 6
param['silent'] = 1
param['nthread'] = 4
param['num_class'] = num_class

watchlist = [ (xg_train,'train'), (xg_test, 'test') ]
num_round = 5
bst = xgb.train(param, xg_train, num_round, watchlist );
# get prediction
pred = bst.predict( xg_test );

print ('predicting, classification error=%f' % (sum( int(pred[i]) != test_Y[i] for i in range(len(test_Y))) / float(len(test_Y)) ))

# do the same thing again, but output probabilities
param['objective'] = 'multi:softprob'
bst = xgb.train(param, xg_train, num_round, watchlist );
# Note: this convention has been changed since xgboost-unity
# get prediction, this is in 1D array, need reshape to (ndata, nclass)
yprob = bst.predict( xg_test ).reshape( test_Y.shape[0],  num_class)
ylabel = np.argmax(yprob, axis=1)

#import ml_metrics as metrics
#target = [[l] for l in t2["hotel_cluster"]]
#metrics.mapk(target, predictions, k=5)
print ('predicting, classification error=%f' % (sum( int(ylabel[i]) != test_Y[i] for i in range(len(test_Y))) / float(len(test_Y)) ))